# Bird images classification y deep learning
The first version was with a dataset ready to exploit but with only 3 species. So I found "Avibase" that is a world bird database. We can't to their database but they have a flickr where their community can post birds pictures with precise tags.

## 1. Downloading data

As we don't have access to the flickr database we need to found a way to get data. A python library exist to download flickr image. To identify pictures we gonna use tags that represents names of birds species. To finds these tags we gonna use Avibase and make a tags list. first We load the tags list.

In [7]:
filePath = r"tags_list.txt"
with open(filePath, 'r', encoding='utf-8') as f: 	
    lines = f.readlines()
    f.close()

lines = list(map(lambda each:each.strip("\n"), lines))
print(lines)

['Spatula querquedula', 'creagrus furcatus', 'Cygnus columbianus']


Now that we have loaded tags we can find images urls and save it in a key value array.

In [8]:
import flickrapi
import urllib
from PIL import Image

# Flickr api access key 
flickr=flickrapi.FlickrAPI('c6a2c45591d4973ff525042472446ca2', '202ffe6f387ce29b', cache=True)

photos = {}
urls = {}

for tag in lines:
    print(tag)
    photos[tag] = flickr.walk(text=tag,
                        tag_mode='all',
                        tags=tag,
                        extras='url_c',
                        per_page=100,           # may be you can try different numbers..
                        sort='relevance')

    urls[tag] = []
    for i, photo in enumerate(photos[tag]):
        print (i)
        
        url = photo.get('url_c')
        urls[tag].append(url)
        
        # get 50 urls
        if i > 800:
            break

    print (urls)


Spatula querquedula
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
27

Before downloading images we need to split them in training and validation batch.

In [9]:
from sklearn.model_selection import train_test_split

X_train = []
X_val = []
for tag in lines:
    trains ,vals=train_test_split(urls[tag],test_size=0.2)
    for train in trains:
        if train is not None:
            X_train.append([tag, train])
    for val in vals:
        if val is not None:
            X_val.append([tag, val])

print('validation length : ', len(X_val))
print('training length : ', len(X_train))

validation length :  421
training length :  1652


In [10]:
import os 

if not os.path.exists('train'):
    os.mkdir('train')
if not os.path.exists('val'):
  os.mkdir('val')

for i in range(0, len(X_train)):
  path = r"train/" + X_train[i][0]
  if not os.path.exists(path):
    os.mkdir(path)
  urllib.request.urlretrieve(X_train[i][1],"train/"+X_train[i][0]+"/image"+str(i)+'.jpg')
  print('Téléchargement réussi train ' +str(i))

for i in range(0, len(X_val)):
  path = r"val/" + X_val[i][0]
  if not os.path.exists(path):
    os.mkdir(path)
  urllib.request.urlretrieve(X_val[i][1],"val/"+X_val[i][0]+"/image"+str(i)+'.jpg')
  print('Téléchargement réussi val ' +str(i))



Téléchargement réussi train 0
Téléchargement réussi train 1
Téléchargement réussi train 2
Téléchargement réussi train 3
Téléchargement réussi train 4
Téléchargement réussi train 5
Téléchargement réussi train 6
Téléchargement réussi train 7
Téléchargement réussi train 8
Téléchargement réussi train 9
Téléchargement réussi train 10
Téléchargement réussi train 11
Téléchargement réussi train 12
Téléchargement réussi train 13
Téléchargement réussi train 14
Téléchargement réussi train 15
Téléchargement réussi train 16
Téléchargement réussi train 17
Téléchargement réussi train 18
Téléchargement réussi train 19
Téléchargement réussi train 20
Téléchargement réussi train 21
Téléchargement réussi train 22
Téléchargement réussi train 23
Téléchargement réussi train 24
Téléchargement réussi train 25
Téléchargement réussi train 26
Téléchargement réussi train 27
Téléchargement réussi train 28
Téléchargement réussi train 29
Téléchargement réussi train 30
Téléchargement réussi train 31
Téléchargement réu

We import libraries needed to generate a model

In [ ]:
import tensorflow as tf
from tensorflow import keras
import keras
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from sklearn.model_selection import train_test_split


from tensorflow.keras.optimizers import Adam 
from keras.metrics import categorical_crossentropy
from keras.models import load_model
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

We import images

In [ ]:
train_path="train"
test_path="val"

In [ ]:
data_gen = dict(rescale=1./255, 
                rotation_range=35, 
                zoom_range = 0.3,
                shear_range=0.2,
                channel_shift_range=0.3,
                fill_mode='nearest',
                cval=0.3,
                width_shift_range=0.2,
                height_shift_range=0.2, 
                horizontal_flip = 'true',
                featurewise_std_normalization=True,
                samplewise_center=True,
                samplewise_std_normalization=True,
                zca_whitening=True,
                zca_epsilon=25,
                brightness_range=0.3,
                vertical_flip=True
                )

train_datagen = ImageDataGenerator(data_gen)

test_datagen = ImageDataGenerator(data_gen)

In [ ]:
train_generator = train_datagen.flow_from_directory(train_path, 
                                                    target_size=(224, 224), 
                                                    batch_size=128, 
                                                    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test_path, 
                                                    target_size=(224, 224), 
                                                    batch_size=128, 
                                                    class_mode='categorical')

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(224, 224, 3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (1, 1), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(256, (1, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
#model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Dense(256))
model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(128))
model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(32))
#model.add(Dropout(0.5))
model.add(Dense(16))
#model.add(Dropout(0.5))
model.add(Dense(8))
#model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

In [ ]:
ad = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss='categorical_crossentropy',
              optimizer=ad,
              metrics=['accuracy'])

In [ ]:
with tf.device('/device:GPU:0'):
    model.fit(train_generator, 
                    shuffle=True,
                    epochs=17, 
                    validation_data=test_generator)